In [ ]:
#yes~~!

In [ ]:
class ActorModel:
    def __init__(self,state_size,action_size, tau):
        self.state_size = state_size
        self.action_size = action_size
        
        self.tau = tau
        
        self.actor = self.build_actor()
        self.target_actor = self.build_actor()
        
    
    def build_actor(self):
        input_layer = Input(shape =(self.state_size,))
        h1 = Dense(24, activation ='relu',kernel_initializer='he_uniform')(input_layer)
        h2 = Dense(24, activation ='relu',kernel_initializer='he_uniform')(h1)
        h3 = Dense(self.action_size, activation = 'relu',kernel_initializer='he_uniform')(h2)
        output = Dense(self.action_size, activation='softmax',kernel_initializer='he_uniform')(h3)
        model = Model(inputs =input_layer, outputs = output)
        return model,input_layer
    
    def get_action(self,state):
        if(random.random()< self.epsilon):
            if(self.epsilon > 0.01):
                self.epsilon = self.epsilon * self.decaying_epsilon
            
            return randint((self.action_size))
        
        policy= self.target_actor.predict(state) #.flatten()
        
        return np.random.choice(self.action_size,1,p=policy[0])[0]
    
    def actor_optimizer(self):
        action = K.placeholder(shape =[None, self.action_size])
        advantage = K.placeholder(shape=[None,])
        action_prob = K.sum(action * self.actor.output,axis = 1)
        cross_entropy = K.log(action_prob) * advantage
        loss = -K.sum(cross_entropy) #최대화하려면 마이너스?
        
        optimizer = Adam(lr=self.actor_lr)
        updates = optimizer.get_updates(self.actor.trainable_weights,[],loss)
        train = K.function([self.actor.input,action,advantage],[],updates=updates)
        return train
    
    def target_actor_train(self):
        actor_weights = self.actor.get_weights()
        actor_target_weights = self.target_actor.get_weights()
        for i in range(len(actor_weights)):
            actor_target_weights[i] = self.tau * actor_weights[i] + (1-self.tau) \
            * actor_target_weights[i]
        self.target_actor.set_weights(actor_target_weights)

In [ ]:
target_q_values = critic.target_model.predict(
                \[new_states, actor.target_model.predict(new_states)]) 
a_for_grad = actor.model.predict(states)
grads = critic.gradients(states, a_for_grad)
actor.train(states, grads)
actor.target_train()
critic.target_train()